In [17]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating new project at `~/Documents/eth_courses/notebooks/others`
  No Changes to `~/Documents/eth_courses/notebooks/others/Project.toml`
  No Changes to `~/Documents/eth_courses/notebooks/others/Manifest.toml`


In [24]:
# Pkg.add("TreeView")

## @show macro

In [16]:
let

x::Float64 = 64.0

function myshow(x::Float64)
    println("x= " , x);
end

myshow(x) # custom  function 

@show x #macro

# show generated code
Base.remove_linenums!(@macroexpand @show x) 
end

x= 64.0
x = 64.0


quote
    Base.println("x = ", Base.repr(begin
                local var"#35#value" = x
            end))
    var"#35#value"
end

 ## Expression tree (computational graph)

In [18]:
ex = :(x + y * z)

:(x + y * z)

In [19]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol y
        3: Symbol z


In [26]:
using TreeView

@tree x + y * z

LabelledTree(Graphs.SimpleGraphs.SimpleDiGraph{Int64}(4, [[2, 3], Int64[], [4, 5], Int64[], Int64[]], [Int64[], [1], [1], [3], [3]]), Any[:+, :x, :*, :y, :z])